In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18213043996116014587, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12692745579004147398
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from collections import Counter
import itertools
import shutil
import random
import glob
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

In [5]:
!pip install python-chess

In [6]:
!pip install "python_chess==0.30.1"

     |████████████████████████████████| 133kB 9.3MB/s 
  Found existing installation: python-chess 0.23.11
    Uninstalling python-chess-0.23.11:
      Successfully uninstalled python-chess-0.23.11


In [7]:
import chess
import chess.pgn

In [8]:
from google.colab import drive
drive.mount('/content/Pdrive')

Mounted at /content/Pdrive


In [9]:
find_num = {'a1': chess.A1, 'a2': chess.A2, 'a3': chess.A3, 'a4': chess.A4, 'a5': chess.A5, 'a6': chess.A6, 'a7': chess.A7,
            'a8': chess.A8, 'b1': chess.B1, 'b2': chess.B2, 'b3': chess.B3, 'b4': chess.B4, 'b5': chess.B5, 'b6': chess.B6,
            'b7': chess.B7, 'b8': chess.B8, 'c1': chess.C1, 'c2': chess.C2, 'c3': chess.C3, 'c4': chess.C4, 'c5': chess.C5,
            'c6': chess.C6, 'c7': chess.C7, 'c8': chess.C8, 'd1': chess.D1, 'd2': chess.D2, 'd3': chess.D3, 'd4': chess.D4, 
            'd5': chess.D5, 'd6': chess.D6, 'd7': chess.D7, 'd8': chess.D8, 'e1': chess.E1, 'e2': chess.E2, 'e3': chess.E3,
            'e4': chess.E4, 'e5': chess.E5, 'e6': chess.E6, 'e7': chess.E7, 'e8': chess.E8, 'f1': chess.F1, 'f2': chess.F2,
            'f3': chess.F3, 'f4': chess.F4, 'f5': chess.F5, 'f6': chess.F6, 'f7': chess.F7, 'f8': chess.F8, 'g1': chess.G1,
            'g2': chess.G2, 'g3': chess.G3, 'g4': chess.G4, 'g5': chess.G5, 'g6': chess.G6, 'g7': chess.G7, 'g8': chess.G8,
            'h1': chess.H1, 'h2': chess.H2, 'h3': chess.H3, 'h4': chess.H4, 'h5': chess.H5, 'h6': chess.H6, 'h7': chess.H7,
            'h8': chess.H8}
        

In [10]:
def data_augment(list_in, str_from): 
    
    # augments the board position when it's black's turn to make it appear as if it's white's turn.
    # for this, we just need to rotate the board by 180 degrees anti-clock and flip the color of all the pieces.
    
    list_out = ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']
    
    var = 7
    
    if len(list_in) == 8:
        
        for el in list_in:
            el_rev = el[::-1]
            fin = ""
            for char in el_rev:
                if char.isalpha():
                    if char.islower():
                        fin += char.upper()
                    elif char.isupper():
                        fin += char.lower()
                elif char.isdigit():
                    fin += char
            
            list_out[var] = fin
            var = var - 1
              
    else:
        print('Error: List not compatible')
    
    fin1 = ""
    
    alpha = list(str_from)
    fin1 += chr(8 - ord(alpha[0]) + ord('a') -1 + 97)
    fin1 += str(9 - int(alpha[1]))
    num = find_num[fin1]
    
    return list_out, num

In [11]:
def count_total_moves(game):
    
    # counts total number of moves of any game.
    
    count = 0
    for posn in game.mainline():
        count += 1 
    
    return count

In [12]:
def fen_to_matrix(string_fen, string_move):
    
    # for a given board in FEN notation, it converts that to a corresponding 8x8x6 matrix board along with it's output label.
    # The output label can be the position from where move was made or the position to where the move was made. It'll be a
    # number between 0 to 63 corresponding to the given string- string_move.
    
    board_matrix = np.zeros((8, 8, 6))
    
    fen_list = string_fen.split(" ")[0].split("/")
    val_ = 0
    
    for x in fen_list:
        temp_s = ""
        for y in x:
            if y.isalpha():
                temp_s += y
            elif y.isdigit():
                for al_ in range(int(y)):
                    temp_s += y
                    
        fen_list[val_] = temp_s
        val_ += 1  
        
    next_ = string_fen.split(" ")[1]
    
    
    
    if next_ == "w":
        label = find_num[string_move]
        
        for i in range(len(fen_list)):
            for j in range(len(fen_list[i])):
                if fen_list[i][j].isalpha():
                    board_matrix[i, j, notation[fen_list[i][j]][0]] = notation[fen_list[i][j]][1]    
    elif next_ == "b":
        fen_list, label = data_augment(fen_list, string_move)
        
        for i in range(len(fen_list)):
            for j in range(len(fen_list[i])):
                if fen_list[i][j].isalpha():
                    board_matrix[i, j, notation[fen_list[i][j]][0]] = notation[fen_list[i][j]][1]
                    
    
    return board_matrix, label

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from collections import Counter

In [14]:
def encoding_label(y_in):
    
    # encodes the output vector (from 0 to 63 in our case) 
    
    encoder = LabelEncoder()
    encoder.fit(y_in)
    encoded_y = encoder.transform(y_in)
    
    return encoded_y

In [15]:
def tt_split(Xin, yin):
    
    # does train-test split using stratified KFold cross validation method.
    
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)    # 80% - 20% split.
    
    skf.get_n_splits(Xin, yin)
    
    for train_index, test_index in skf.split(Xin, yin):
        X_train, X_test = Xin[train_index], Xin[test_index]
        y_train, y_test = yin[train_index], yin[test_index]
        
    return X_train, X_test, y_train, y_test

In [16]:
path = '/content/Pdrive/MyDrive/Colab Notebooks/FICS_Database.pgn'
file_op = open(path)

In [17]:
list_games = []

while True:
    readd = chess.pgn.read_game(file_op)
    if readd is None:
        break
    
    list_games.append(readd)

In [18]:
print(len(list_games))  # total 5794 games have been taken as the dataset. Each game would contain 60-160 moves on an avg.

5794


In [19]:
notation = {'p': (0, -1),
            'n': (1, -1),
            'b': (2, -1),
            'r': (3, -1),
            'q': (4, -1),
            'k': (5, -1),
            'P': (0, 1),
            'N': (1, 1),
            'B': (2, 1),
            'R': (3, 1),
            'Q': (4, 1),
            'K': (5, 1)
             }

In [20]:
# preparing dataset for the 'to CNN'

boards_list_to = []
moves_list_to = []

for gm_to in list_games:
    reg1_to = 0
    reg2_to = 0
    countt_to = count_total_moves(gm_to)
    moves_list_temp_to = []
    
    for pos_to in gm_to.mainline():
        reg1_to += 1
        if reg1_to!=1:
            moves_list_temp_to.append(str(pos_to.move)[2] + str(pos_to.move)[3]) # the position was appended to where the move
                                                                                                                   # was made
    for posi_to in gm_to.mainline():
        reg2_to += 1
        if reg2_to!=countt_to:
            gamma0_to, gamma1_to = fen_to_matrix(posi_to.board().fen(), moves_list_temp_to[reg2_to-1])
            boards_list_to.append(gamma0_to)
            moves_list_to.append(gamma1_to)
                

In [21]:
y_toCNN = np.array(moves_list_to)

In [22]:
# Encoding the labels (or the possible outputs) using the above-made function.

y_test_to = encoding_label(y_toCNN)

In [23]:
# Train-Test split:

X_train_to, X_test_to, y_train_to, y_test_to = tt_split(np.array(boards_list_to), y_test_to)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [27]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (2, 2), activation = 'elu', kernel_regularizer = l2(0.000005), input_shape = (8, 8, 6)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'elu', kernel_regularizer = l2(0.000005)))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'elu', kernel_regularizer = l2(0.000005)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(units = 64, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 7, 7, 32)          800       
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 64)          18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 5, 5, 64)          256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 128)        

In [28]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x = X_train_to, y = y_train_to, batch_size = 128, epochs = 60)

Epoch 1/60
5041/5041 [==============================] - 24s 5ms/step - loss: 3.9375 - accuracy: 0.0791
Epoch 2/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.4242 - accuracy: 0.1546
Epoch 3/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.2619 - accuracy: 0.1791
Epoch 4/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.1780 - accuracy: 0.1907
Epoch 5/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.1213 - accuracy: 0.1995
Epoch 6/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.0851 - accuracy: 0.2041
Epoch 7/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.0567 - accuracy: 0.2086
Epoch 8/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.0321 - accuracy: 0.2126
Epoch 9/60
5041/5041 [==============================] - 23s 5ms/step - loss: 3.0096 - accuracy: 0.2160
Epoch 10/60
5041/5041 [==============================] - 23s 5ms/step - l

In [29]:
model.evaluate(X_test_to, y_test_to, batch_size = 128)

1261/1261 [==============================] - 4s 3ms/step - loss: 2.6837 - accuracy: 0.2722


[2.6836605072021484, 0.2722448408603668]